In [ ]:
!pip install climetlab climetlab-maelstrom-power-production scikit-learn matplotlib

In [ ]:
import climetlab as cml

import utils

## Load data

In [ ]:
constants_dataset = cml.load_dataset("maelstrom-constants-a-b")
production_dataset = cml.load_dataset("maelstrom-power-production", wind_turbine_id=30)
model_level_dataset = cml.load_dataset("maelstrom-weather-model-level", date="2019-01-01")
surface_dataset = cml.load_dataset("maelstrom-weather-surface-level", date="2019-01-01")
constants = constants_dataset.to_xarray()
production = production_dataset.to_xarray()
model_level_data = model_level_dataset.to_xarray()
surface_data = surface_dataset.to_xarray()

## Feature engineering
### Find the grid point closest to the coordinates of the wind turbine

In [ ]:
closest_grid_point = utils.get_closest_grid_point_to_wind_turbine(
    production_data=production,
    data=model_level_data,
)

### Calculate the air density at the model level

In [ ]:
model_level = 133  # is roughly at the wind turbine's hub height (100 m)
air_density = utils.calculate_air_density(
    grid_point=closest_grid_point,
    model_level=model_level,
    model_level_data=model_level_data,
    surface_data=surface_data,
    constants=constants,
)

### Calculate the absolute wind speed at the model level and the wind direction

In [ ]:
absolute_wind_speed, wind_direction = utils.calculate_absolute_wind_speed_and_wind_direction(
    grid_point=closest_grid_point,
    model_level=model_level,
    model_level_data=model_level_data,
)

### Additional features

Passing the time of the day and year improves the model and, as a result, forecast quality.

Examples:

Time of day:

- 2020-01-01T00:00:00 corresponds to 0.0
- 2020-01-01T06:00:00 corresponds to 0.25
- 2020-01-01T12:00:00 corresponds to 0.5
- 2020-01-01T18:00:00 corresponds to 0.75
- and so forth...

Time of year:

- 2020-01-01T00:00:00 corresponds to 0.0
- 2020-07-02T00:00:00 corresponds to roughly 0.5
- 2020-12-31T00:00:00 corresponds to 1.0
- and so forth...


In [ ]:
dates = utils.get_dates_from_time_coordinate(model_level_data)
time_of_day, time_of_year = utils.get_time_of_day_and_year(dates)

### Resampling of the production data
Since the production data are available in 10-minute intervals, they need to be resampled to match the hourly model data. The production data may also contain negative values. These result e.g. when a wind turbine is moved due to lack of wind. Hence, we set them 0 to increase the model quality.

In [ ]:
prod_resampled = utils.resample_and_clear_production_data_to_hourly_timeseries(
    production, dates=dates
)

## Train the model
Input samples (features) are:

- air density
- absolute wind speed
- wind direction
- time of day
- time of year

Target values are the production data.

In [ ]:
model = utils.train_model(
    air_density=air_density,
    absolute_wind_speed=absolute_wind_speed,
    wind_direction=wind_direction,
    time_of_day=time_of_day,
    time_of_year=time_of_year,
    production=prod_resampled,
)

## Create a forecast
### Get and convert the weather data

In [ ]:
model_level_dataset = cml.load_dataset("maelstrom-weather-model-level", date="2019-01-02")
surface_dataset = cml.load_dataset("maelstrom-weather-surface-level", date="2019-01-02")
model_level_data = model_level_dataset.to_xarray()
surface_data = surface_dataset.to_xarray()

### Calculate the necessary features
The features used for the forecast have to be identical to those used for training the model.

In [ ]:
air_density = utils.calculate_air_density(
    grid_point=closest_grid_point,
    model_level=model_level,
    model_level_data=model_level_data,
    surface_data=surface_data,
    constants=constants,
)

absolute_wind_speed, wind_direction = utils.calculate_absolute_wind_speed_and_wind_direction(
    grid_point=closest_grid_point,
    model_level=model_level,
    model_level_data=model_level_data,
)

dates = utils.get_dates_from_time_coordinate(model_level_data)
time_of_day, time_of_year = utils.get_time_of_day_and_year(dates)

### Predict the power production

In [ ]:
production_forecast = utils.predict_power_production(
    model,
    air_density=air_density,
    absolute_wind_speed=absolute_wind_speed,
    wind_direction=wind_direction,
    time_of_day=time_of_day,
    time_of_year=time_of_year,
)

## Evaluate the forecast quality
### Get the real production data for the forecast time range

In [ ]:
production_real = utils.resample_and_clear_production_data_to_hourly_timeseries(
    production, dates=dates
)

### Compare the forecast and the real production data
#### Plot the forecasted and real power production

In [ ]:
utils.plot_forecast_and_real_production_data(
    indexes=dates,
    forecast=production_forecast,
    real=production_real,
)

#### Calculate the NMAE

In [ ]:
power_rating = utils.get_power_rating(production)
nmae = utils.normalized_mean_absolute_error(
    production_real, 
    production_forecast, 
    normalization=power_rating,
)
print(f"NMAE: {nmae}")